# NDVI over Sentinel-2 imagery in the focus region

In [ ]:
import geopandas as gpd
import rasterio as rio
from pathlib import Path
from shapely.geometry import box

# this script directory in jupyterlab
script_dir = Path.cwd()

In [ ]:
def load_focus_region() -> gpd.GeoDataFrame:
    """
    Load the shapefile for the focus region
    Author: jan mattis töller

    Args:
        None
    Returns:
        shapefile_focus_region (gpd.GeoDataFrame): The shapefile for the focus region
    """
    shapefile_path = script_dir / "data" / "source" / "frontlinie.shp"
    shapefile_focus_region = gpd.read_file(shapefile_path)

    assert isinstance(shapefile_focus_region, gpd.GeoDataFrame), "shapefile_focus_region is not a GeoDataFrame"
    return shapefile_focus_region

focus_region = load_focus_region()
print(focus_region)

AttributeError: module 'rasterio' has no attribute 'shutil'

In [ ]:
def load_geotiff(path: Path) -> gpd.GeoDataFrame:
    """
    Load the geotiff file as a rasterio object
    Args: 
        path (Path): Path to the geotiff file
    Returns: 
        geodataframe (gpd.GeoDataFrame): The geopandas GeoDataFrame
    """
    with rio.open(path) as src:
        # Read the geotiff file as a numpy array
        data = src.read()
        meta = src.meta

        geodataframe = gpd.GeoDataFrame(
            data=data,
            crs=meta['crs'],
            geometry=[box(*src.bounds)],
        )
    
    assert isinstance(geodataframe, gpd.GeoDataFrame), "geodataframe is not a GeoDataFrame"
    return geodataframe

In [14]:
def index_NDVI(geodataframe: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Index the NDVI
    Args:
        geodataframe (gpd.GeoDataFrame): The geopandas GeoDataFrame
    Returns:
        indexed_geodataframe (gpd.GeoDataFrame): The indexed geopandas GeoDataFrame
    """
    # Assuming the GeoDataFrame has columns 'NIR' and 'Red' for the respective bands
    if 'NIR' not in geodataframe.columns or 'Red' not in geodataframe.columns:
        raise ValueError("GeoDataFrame must contain 'NIR' and 'Red' columns")

    # Calculate NDVI
    geodataframe['NDVI'] = (geodataframe['NIR'] - geodataframe['Red']) / (geodataframe['NIR'] + geodataframe['Red'])

    assert isinstance(geodataframe, gpd.GeoDataFrame), "indexed_geodataframe is not a GeoDataFrame"
    return geodataframe

In [ ]:
def calculate_raster_difference(geodataframe1: gpd.GeoDataFrame, geodataframe2: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Calculate the difference between two rasters
    Args:
        geodataframe1 (gpd.GeoDataFrame): The first geopandas GeoDataFrame
        geodataframe2 (gpd.GeoDataFrame): The second geopandas GeoDataFrame
    Returns:
        difference_geodataframe (gpd.GeoDataFrame): The difference geopandas GeoDataFrame
    """
    # Ensure both GeoDataFrames have the same columns
    if not geodataframe1.columns.equals(geodataframe2.columns):
        raise ValueError("GeoDataFrames must have the same columns")

    # Calculate the difference for each column except 'geometry'
    difference_data = {}
    for column in geodataframe1.columns:
        if column != 'geometry':
            difference_data[column] = geodataframe1[column] - geodataframe2[column]

    # Create a new GeoDataFrame with the difference data
    difference_geodataframe = gpd.GeoDataFrame(difference_data, 
                                               crs=geodataframe1.crs, 
                                               geometry=geodataframe1.geometry)

    assert isinstance(difference_geodataframe, gpd.GeoDataFrame), "difference_geodataframe is not a GeoDataFrame"
    return difference_geodataframe

In [ ]:
# Hauptskript

# Load the focus region
shapefile_focus_region = load_focus_region()

# Load the geotiff file as a rasterio object
raster_old = load_geotiff(Path("path_to_geotiff_file_old"))
raster_new = load_geotiff(Path("path_to_geotiff_file_new"))

# Calculate the NDVI indexs 
NDVI_old = index_NDVI(raster_old)
NDVI_new = index_NDVI(raster_new)

# Calculate the difference between the two rasters
difference = calculate_raster_difference(NDVI_old, NDVI_new)

print("Success!")